In [14]:
import pandas as pd
import numpy as np

In [15]:
pse_annual_financials = pd.read_excel("PSE Companies Financial Reports.xlsx", sheet_name="Annual Financial Reports")
pse_quarterly_financials = pd.read_excel("PSE Companies Financial Reports.xlsx", sheet_name="Quarterly Financial Reports")

# ANNUAL

In [17]:
# Set pandas display option to show numbers with 2 decimal places
pd.set_option('display.float_format', '{:,.2f}'.format)  # 2 decimals, commas for thousands

# Function to clean numbers: remove commas, handle parentheses as negatives, handle dashes
def clean_number(x):
    x = str(x).replace(',', '').strip()         # remove commas and spaces
    if x in ['', '-', 'nan', 'NaN']:            # treat missing or dash as 0
        return 0.0
    if x.startswith('(') and x.endswith(')'):   # convert (123) to -123
        x = '-' + x[1:-1]
    try:
        return float(x)
    except ValueError:
        return 0.0   # fallback if any unexpected string appears

# Apply cleaning function
pse_annual_financials['Current Year'] = pse_annual_financials['Current Year'].apply(clean_number)
pse_annual_financials['Previous Year'] = pse_annual_financials['Previous Year'].apply(clean_number)

# Pivot to wide format
pse_annual_financials_clean = pse_annual_financials.pivot_table(
    index=['Company Name', 'Fiscal Period'],   # row per company per year
    columns='Item',                            # each financial metric becomes a column
    values=['Current Year', 'Previous Year']   # numeric values
)

# Flatten MultiIndex columns
pse_annual_financials_clean.columns = [f'{metric}_{year}' for year, metric in pse_annual_financials_clean.columns]
pse_annual_financials_clean = pse_annual_financials_clean.reset_index()

# Preview the cleaned, wide-format DataFrame with 2 decimals
pse_annual_financials_clean

,Company Name,Fiscal Period,Book Value Per Share_Current Year,Current Assets_Current Year,Current Liabilities_Current Year,Earnings/(Loss) Per Share (Basic)_Current Year,Earnings/(Loss) Per Share (Diluted)_Current Year,Gross Expense_Current Year,Gross Revenue_Current Year,Income/(Loss) Before Tax_Current Year,...,Gross Expense_Previous Year,Gross Revenue_Previous Year,Income/(Loss) Before Tax_Previous Year,Net Income/(Loss) After Tax_Previous Year,Net Income/(Loss) Attributable to Parent_Previous Year,Retained Earnings/(Deficit)_Previous Year,Stockholders' Equity_Previous Year,Stockholders' Equity - Parent_Previous Year,Total Assets_Previous Year,Total Liabilities_Previous Year
0,A. SORIANO CORPORATION,"Dec 31, 2024",22.05,"23,823,216,771.00","2,721,553,027.00",3.81,3.81,"11,903,410,670.00","16,817,732,782.00","5,174,483,163.00",...,"10,742,606,738.00","13,798,487,906.00","3,017,195,730.00","2,649,195,685.00","2,552,017,982.00","21,346,742,307.00","23,987,967,990.00","23,172,977,333.00","27,692,602,665.00","3,704,634,675.00"
1,"ABACORE CAPITAL HOLDINGS, INC.","Dec 31, 2024",5.01,"684,939,635.00","1,525,237,968.00",0.60,0.60,"163,291,904.00","3,457,155,961.00","3,293,864,057.00",...,"163,694,824.00","3,028,532,061.00","2,864,837,237.00","2,435,550,871.00","2,451,443,075.00","14,090,500,587.00","19,128,699,840.00","6,352,566,583.00","23,288,638,023.00","4,159,938,183.00"
2,"ABOITIZ EQUITY VENTURES, INC.","Dec 31, 2024",51.01,"212,499,004.00","132,646,108.00",3.25,3.25,"252,232,112.00","302,825,653.00","48,543,017.00",...,"272,574,405.00","310,619,089.00","51,514,642.00","42,921,207.00","23,546,289.00","227,276,219.00","375,346,444.00","271,954,731.00","833,874,659.00","458,528,215.00"
3,ABOITIZ POWER CORPORATION,"Dec 31, 2024",28.20,"123,026,353.00","78,985,995.00",4.70,4.70,"156,297,345.00","197,492,257.00","45,942,946.00",...,"175,314,818.00","207,099,897.00","43,742,299.00","36,465,434.00","33,101,720.00","167,396,036.00","193,974,365.00","180,138,066.00","487,023,306.00","293,048,941.00"
4,ACEN CORPORATION,"Dec 31, 2024",3.72,"59,132,109.00","29,670,253.00",0.19,0.19,"34,341,343.00","37,300,402.00","10,942,397.00",...,"39,200,966.00","36,499,133.00","9,388,972.00","9,106,674.00","7,396,140.00","24,871,807.00","173,375,857.00","143,472,317.00","284,933,500.00","111,557,643.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,"VICTORIAS MILLING COMPANY, INC.","Aug 31, 2024",4.51,"5,389,875.00","1,227,708.00",0.57,0.57,"10,204,703.00","11,377,071.00","1,724,989.00",...,"13,219,888.00","15,546,991.00","1,723,593.00","1,570,672.00","1,571,839.00","8,354,491.00","11,180,308.00","11,178,292.00","13,543,683.00","2,363,375.00"
100,VITARICH CORPORATION,"Dec 31, 2024",0.74,"3,206,286,572.00","3,128,802,760.00",0.07,0.07,"12,141,471,258.00","12,539,289,231.00","314,383,734.00",...,"12,485,252,234.00","12,495,914,596.00","6,580,697.00","13,304,916.00","13,304,916.00","303,502,861.00","1,901,342,078.00","1,901,342,078.00","5,222,496,068.00","3,321,153,990.00"
101,VIVANT CORPORATION,"Dec 31, 2024",19.60,"8,195,538,164.00","3,413,758,867.00",2.30,0.00,"8,924,332,459.00","12,201,703,051.00","3,003,800,074.00",...,"2,724,883,383.00","8,264,817,155.00","2,578,413,102.00","2,405,542,792.00","2,295,804,486.00","17,444,734,635.00","19,649,660,001.00","18,292,285,621.00","30,038,390,747.00","10,388,730,746.00"
102,"WELLEX INDUSTRIES, INCORPORATED","Dec 31, 2024",0.43,"60,349,587.00","14,234,212.00",0.06,0.06,"32,253,681.00","32,563,156.00","201,543,945.00",...,"29,727,138.00","25,749,089.00","8,189,583.00","3,605,543.00","3,605,543.00","-2,096,773,807.00","1,203,754,631.00","1,203,754,631.00","1,623,616,657.00","419,862,026.00"


# QUARTERLY

In [19]:
# Set pandas display option to show numbers with 2 decimals and commas
pd.set_option('display.float_format', '{:,.2f}'.format)

# Function to clean numbers: remove commas, handle parentheses as negatives, handle dashes
def clean_number(x):
    x = str(x).replace(',', '').replace(' ', '').strip()  # remove commas, spaces
    if x in ['', '-', 'nan', 'NaN']:                      # treat missing or dash as 0
        return 0.0
    if x.startswith('(') and x.endswith(')'):             # convert (123) to -123
        x = '-' + x[1:-1]
    try:
        return float(x)
    except ValueError:
        return 0.0  # fallback for any unexpected string

# List all columns to clean (numeric columns)
numeric_cols = ['Current Period', 'Previous Period', 'Current 3 Months', 
                'Previous 3 Months', 'Current YTD', 'Previous YTD']

# Apply cleaning function to all numeric columns
for col in numeric_cols:
    pse_quarterly_financials[col] = pse_quarterly_financials[col].apply(clean_number)

# Pivot to wide format
pse_quarterly_financials_clean = pse_quarterly_financials.pivot_table(
    index=['Company Name', 'Fiscal Period'],  # row per company per fiscal period
    columns='Item',                           # each financial metric becomes a column
    values=numeric_cols                       # include all numeric columns
)

# Flatten MultiIndex columns: combine metric and period
pse_quarterly_financials_clean.columns = [f'{metric}_{period}' for period, metric in pse_quarterly_financials_clean.columns]
pse_quarterly_financials_clean = pse_quarterly_financials_clean.reset_index()

# Preview cleaned, wide-format quarterly DataFrame
pse_quarterly_financials_clean

,Company Name,Fiscal Period,Book Value Per Share_Current 3 Months,Current Assets_Current 3 Months,Current Liabilities_Current 3 Months,Earnings/(Loss) Per Share (Basic)_Current 3 Months,Earnings/(Loss) Per Share (Diluted)_Current 3 Months,Gross Expense_Current 3 Months,Gross Revenue_Current 3 Months,Income/(Loss) Before Tax_Current 3 Months,...,Gross Expense_Previous YTD,Gross Revenue_Previous YTD,Income/(Loss) Before Tax_Previous YTD,Net Income/(Loss) After Tax_Previous YTD,Net Income/(Loss) Attributable to Parent_Previous YTD,Retained Earnings/(Deficit)_Previous YTD,Stockholders' Equity_Previous YTD,Stockholders' Equity - Parent_Previous YTD,Total Assets_Previous YTD,Total Liabilities_Previous YTD
0,A. SORIANO CORPORATION,"Jun 30, 2025",0.00,0.00,0.00,1.26,1.26,"3,042,895.00","4,682,982.00","1,592,651.00",...,"5,777,996.00","9,431,333.00","3,793,215.00","3,517,238.00","3,433,403.00",0.00,0.00,0.00,0.00,0.00
1,"ABACORE CAPITAL HOLDINGS, INC.","Jun 30, 2025",0.00,0.00,0.00,-0.01,0.00,"29,480,769.00","2,868,846.00","-26,611,923.00",...,"63,104,526.00","132,933,736.00","69,829,210.00","69,829,210.00","75,704,268.00",0.00,0.00,0.00,0.00,0.00
2,"ABOITIZ EQUITY VENTURES, INC.","Jun 30, 2025",0.00,0.00,0.00,0.94,0.94,"64,115,731.00","73,363,756.00","11,742,554.00",...,"127,166,480.00","149,417,835.00","26,241,233.00","21,133,291.00","11,482,563.00",0.00,0.00,0.00,0.00,0.00
3,ABOITIZ POWER CORPORATION,"Jun 30, 2025",0.00,0.00,0.00,1.12,1.12,"40,233,645.00","47,593,866.00","10,266,024.00",...,"81,404,426.00","100,239,455.00","22,763,505.00","18,724,382.00","17,127,669.00",0.00,0.00,0.00,0.00,0.00
4,ACEN CORPORATION,"Sep 30, 2025",0.00,0.00,0.00,0.01,0.01,"7,224,006.00","7,273,027.00","1,177,517.00",...,"24,869,559.00","28,084,895.00","9,569,859.00","8,966,449.00","8,144,445.00",0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,"VICTORIAS MILLING COMPANY, INC.","May 31, 2025",0.00,0.00,0.00,0.07,0.07,"3,836,462.00","4,178,997.00","421,568.00",...,"9,063,591.00","10,048,794.00","1,298,097.00","1,220,375.00","1,219,984.00",0.00,0.00,0.00,0.00,0.00
101,VITARICH CORPORATION,"Jun 30, 2025",0.00,0.00,0.00,0.00,0.00,"2,909,660,703.00","2,924,326,680.00","-9,619,412.00",...,"5,846,640,614.00","6,089,373,398.00","205,059,044.00","167,035,481.00","167,035,481.00",0.00,0.00,0.00,0.00,0.00
102,VIVANT CORPORATION,"Jun 30, 2025",0.00,0.00,0.00,0.66,0.00,"2,179,520,917.00","3,017,650,021.00","764,767,941.00",...,"4,487,886,711.00","5,557,217,453.00","932,650,401.00","871,885,989.00","877,386,235.00",0.00,0.00,0.00,0.00,0.00
103,"WELLEX INDUSTRIES, INCORPORATED","Sep 30, 2025",0.00,0.00,0.00,0.00,0.00,"6,569,554.00","4,998,937.00","-1,487,694.00",...,"24,638,201.00","24,471,671.00","-518,999.00","-518,999.00","-518,999.00",0.00,0.00,0.00,0.00,0.00
